In [1]:
# Basics 
import pandas as pd
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.conversion.log import converter as log_converter
from collections import Counter
from datetime import timedelta
from datetime import datetime
import time
import numpy as np

# Prediction Models
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
# Suffix 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Reshape, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Evaluation
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

#Encoders & scalers 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
log = xes_importer.apply('Road_Traffic_Fine_Management_Process.xes')
df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)

In [2]:
# path = "pm_roads_data.csv"
# df = pd.read_csv(path)

# Baseline

**Data Exploration**

In [4]:
traces = df.groupby('case:concept:name')['concept:name'].apply(list).tolist() # Make a list of all sequences
max_length = df['case:concept:name'].value_counts().max() # Find the longest sequence of events in the dataset
cumulative_counts = df.groupby('case:concept:name').cumcount() # Stores the position in a sequence of each row in the DF
df['time:timestamp'] = pd.to_datetime(df['time:timestamp']) # Converts the timestamp column to datetime

**Event Prediction**

In [5]:
most_common_events = [] # List storing the most common events for each position
for i in range(max_length):
    all_events_at_i = [trace[i] for trace in traces if len(trace) > i] # All events occurring at position i
    counts = Counter(all_events_at_i) # The respective counts per type of event 
    most_common_event = counts.most_common(1)[0][0] # Find the most common event at that position
    most_common_events.append(most_common_event) # Store the most common event in the list
    
most_common_events = most_common_events[1:] # Since we will be predicting only the next event, never the first
most_common_events.append("No next")  # Since after the last event in the longest trace, there is no next one 

**Time Prediction**

In [6]:
times = df.groupby('case:concept:name')['time:timestamp'].apply(list).tolist() # A list of the timestamps of all sequences

average_times = []
for i in range(0, max_length - 1):
    all_times_at_i = [time[i+1] - time[i] for time in times if len(time) > i+1] # All time differences occuring at i
    total_seconds = sum(time.total_seconds() for time in all_times_at_i) # In seconds, otherwise overflow
    average_time = total_seconds / len(all_times_at_i) # Calculate the average for each position
    average_time = timedelta(seconds=average_time) # Revert to time objectcs
    average_times.append(average_time)
    
average_times.append(timedelta()) # Since no event will occur after the last event in the longest trace 

**Add the predictions to the dataframe**

In [7]:
# Baseline results
df["baseline_next_event"] = [most_common_events[i] for i in cumulative_counts] # Next event prediction
df["baseline_time_to_next"] = [average_times[i] for i in cumulative_counts] # Time to next event prediction
df["baseline_time_of_next"] = df["time:timestamp"] + df["baseline_time_to_next"] # Time of next event prediction
df["baseline_time_of_next"] = df["baseline_time_of_next"].dt.strftime('%Y-%m-%d %H:%M:%S+00:00')

**Evaluation of baseline model**

In [8]:
df_eval = df.copy()
df_eval['days_to_next'] = df_eval.groupby('case:concept:name')["time:timestamp"].shift(-1) - df_eval["time:timestamp"]
df_eval['days_to_next'] = df_eval['days_to_next'].dt.days 
df_eval['days_to_next'].fillna(0, inplace = True) 
df_eval['predicted_days'] = df_eval['baseline_time_to_next'].dt.days

df_eval['next_event'] = df_eval.groupby('case:concept:name')['concept:name'].shift(-1)
df_eval['next_event'].fillna('No next', inplace = True)

mse = mean_squared_error(df_eval['predicted_days'], df_eval['days_to_next'])
rmse = np.sqrt(mse)
mae = mean_absolute_error(df_eval['predicted_days'], df_eval['days_to_next'])
f1 = f1_score(df_eval['baseline_next_event'], df_eval['next_event'], average='weighted')

accuracy = accuracy_score(df_eval['baseline_next_event'], df_eval['next_event'])
print('Baseline Evaluation:')
print('Time, MSE:', mse)
print('Time, RMSE:', rmse)
print('Time, MAE:', mae)
print('Events, F1 average:', f1)
print('Events, Accuracy:', accuracy)
del(df_eval)

Baseline Evaluation:
Time, MSE: 18512.763453078525
Time, RMSE: 136.06161638419016
Time, MAE: 75.65790514185976
Events, F1 average: 0.6694420597301748
Events, Accuracy: 0.5723048426451992


In [9]:
# The following objects are no longer used after implementing the baseline and deleting them frees up memory 
del(traces)
del(times)

# Train/Test Splitting

In [3]:
df_model = df.copy() 
df_model['time:timestamp'] = pd.to_datetime(df_model['time:timestamp'])

# Preparing for split
threshold = pd.Timestamp('2009-07-31 00:00:00', tz='UTC') # Set a cut-off timestamp (vertical line)
before_threshold = df_model[df_model['time:timestamp'] < threshold]['case:concept:name'].unique() # Traces that have events before cut-off
after_threshold = df_model[df_model['time:timestamp'] >= threshold]['case:concept:name'].unique() # Traces that have events at/after cut-off
mixed_case_ids = set(before_threshold) & set(after_threshold) # Traces that have both 

In [4]:
# Encode
encoder = LabelEncoder() 
events = df['concept:name'].unique()
events = np.append(events, ['No next', '<SOS>', '<EOS>'])
events
encoder.fit(events)

LabelEncoder()

**Feature Engineering**
Note: the features "dismissal_encoded" and "in_debt" are specific to the RTFM dataset. If running the tool on a different dataset, they can be replaced with other Boolean features or removed. To remove: comment out the part of the code specified in the cell below, remove them as names from the list "features"

In [5]:
#We are going to need this again, so make it into a function: 
def addFeatures(df_model):
    df_model['time:timestamp'] = pd.to_datetime(df_model['time:timestamp']) # Converts the timestamp column to datetime
    
    #The position of an event in a trace
    df_model['position'] = df_model.groupby('case:concept:name').cumcount() # The position of an event in a trace

    #The days that will pass until the next event as integers
    df_model['days_to_next'] = df_model.groupby('case:concept:name')['time:timestamp'].shift(-1) - df_model['time:timestamp']
    df_model['days_to_next'] = df_model['days_to_next'].dt.days # Turns timedelta objects into ints for prediction models
    df_model['days_to_next'].fillna(0, inplace = True)
    df_model['days_to_next'] = df_model['days_to_next'].astype(int)

    #The days that have passed since the last event as integers 
    df_model['days_since_last'] = df_model['time:timestamp'] - df_model.groupby('case:concept:name')['time:timestamp'].shift(1)
    df_model['days_since_last'].fillna(pd.Timedelta(days=0), inplace = True)
    df_model['days_since_last'] = df_model['days_since_last'].dt.days 

    #The current day of the week
    df_model['weekday'] = df_model['time:timestamp'].dt.dayofweek / 7 

    #The days that have passed since the start of the trace
    df_model['days_since_start'] = df_model.groupby('case:concept:name')['days_since_last'].cumsum()

    #The type of the next event that will occur
    df_model['next_event'] = df_model.groupby('case:concept:name')['concept:name'].shift(-1)
    df_model['next_event'].fillna('No next', inplace = True)
    
    df_model['next_event'] = encoder.transform(df_model['next_event'])
    df_model['type_of_event'] = encoder.transform(df_model['concept:name'])

    #DATASET SPECIFIC FEATURES BELOW! If running on a different dataset, comment out the following lines.
    
    #Whether a fine has been dismissed by judge or prefecture appeal 
    dismissal1 = (df_model['dismissal'] == 'G') | (df_model['dismissal'] == '#')
    df_model['dismissal_encoded'] = 0 
    df_model.loc[dismissal1, 'dismissal_encoded'] = 1

    #Whether the person still has some amount to pay
    df_model['debt'] = df_model['amount'].fillna(0) + df_model['expense'].fillna(0) 
    df_model['debt'] = df_model.groupby('case:concept:name')['debt'].cumsum()
    in_debt = (df_model['totalPaymentAmount'].fillna(0) >= df_model['debt'])
    df_model['in_debt'] = 1 #1 if yes
    df_model.loc[in_debt, 'in_debt'] = 0 #0 if fine is completely paid and the case can be closed  
    
    return df_model

In [6]:
df_model = addFeatures(df_model) 

# Note: 'dismissal_encoded' and 'in_debt' are NOT general features, remove if NOT using the RFTM dataset
event_features = ['position', 'type_of_event', 'days_since_last', 'days_since_start', 'dismissal_encoded', 'in_debt']
time_features = ['position', 'type_of_event', 'days_since_last', 'dismissal_encoded']

**Splitting** 

In [7]:
df_train = df_model[(~df_model['case:concept:name'].isin(mixed_case_ids)) & (df_model['case:concept:name'].isin(before_threshold))]
df_test = df_model[(~df_model['case:concept:name'].isin(mixed_case_ids)) & (df_model['case:concept:name'].isin(after_threshold))]
del(df_model)

In [8]:
#Scale based on train data to (hopefully) cover the problem of train traces spanning over longer periods 
scaler = MinMaxScaler()

df_train['days_since_start'] = scaler.fit_transform(np.array(df_train['days_since_start']).reshape(-1,1)) 
df_train['days_since_last'] = scaler.transform(np.array(df_train['days_since_last']).reshape(-1,1)) 

df_test['days_since_last'] = scaler.transform(np.array(df_test['days_since_last']).reshape(-1,1)) 
df_test['days_since_start'] = scaler.transform(np.array(df_test['days_since_start']).reshape(-1,1)) 

**Cleaning incomplete traces from train data (RTFM specific)**

In [9]:
#Clean traces that end in 'Send Fine' 
last = df_train.groupby("case:concept:name").tail(1)
traces_to_clean = last[last["concept:name"] == "Send Fine"]["case:concept:name"].unique()
df_train = df_train[~df_train['case:concept:name'].isin(traces_to_clean)]
#Note that we only do the cleaning on the train data, as filtering the test data would be unrealistic in a real-life situation 

# Training Models (events)

In [17]:
X_train_events = df_train[event_features]
y_train_events = df_train['next_event']

X_test_events = df_test[event_features]
y_test_events = df_test['next_event']

In [18]:
rf = RandomForestClassifier()
rf.fit(X_train_events,y_train_events)

RandomForestClassifier()

# Training Models (time)

In [19]:
x_train_time = df_train[time_features]
y_train_time = df_train['days_to_next']

x_test_time = df_test[time_features]
y_test_time = df_test['days_to_next']

In [20]:
xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42, max_depth = 4)
xgb_reg.fit(x_train_time, y_train_time)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

### Evaluation of random forest on events prediction

In [40]:
# event_prediction = rf.predict(X_test_events)
# f1 = f1_score(y_test_events, event_prediction, average='weighted')
# accuracy = accuracy_score(y_test_events, event_prediction) 

# df_eval = df_test.copy()
# df_eval['predicted_next'] = encoder.inverse_transform(event_prediction)
# df_eval['next_event'] = encoder.inverse_transform(df_eval['next_event'])
# classes = df_eval['next_event'].unique()

# print('Random Forest Evaluation:')
# print('Events, F1 average:', f1)
# print('Events, Accuracy:', accuracy)

Random Forest Evaluation:
Events, F1 average: 0.7314757752339297
Events, Accuracy: 0.7834345274235984


### Evaluation of xgb on time to next event prediction

In [39]:
# time_prediction = xgb_reg.predict(x_test_time)
# mse = mean_squared_error(y_test_time, time_prediction)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test_time, time_prediction)

# print('XGB Evaluation:')
# print('Time, MSE:', mse)
# print('Time, RMSE:', rmse)
# print('Time, MAE:', mae)

XGB Evaluation:
Time, MSE: 7407.458757046154
Time, RMSE: 86.0665948963136
Time, MAE: 38.99964583468667


# Predict on entire dataset to export 

In [23]:
df_predict = df.copy()
df_predict = addFeatures(df_predict)

In [24]:
rf_prediction = rf.predict(df_predict[event_features])
xgb_prediction = xgb_reg.predict(df_predict[time_features])

In [25]:
xgb_prediction = [int(i) for i in xgb_prediction] # Prediction is continious, round down to whole days 
days = pd.to_timedelta(xgb_prediction, unit='D')  # Make them timedelta objects, so we can sum with timestamps

In [26]:
df['rf_next_event'] = encoder.inverse_transform(rf_prediction) # Decode the encoding of the event prediction back into strings
df['xgb_time_of_next'] = df['time:timestamp'] + days # Get the predicted time of next event 

# Suffix Prediction

In [10]:
max_length = 8 # the maximum trace length on which the model will be trained is max_length - 2
no_next = encoder.transform(['No next'])[0]
sos = encoder.transform(['<SOS>'])[0]
eos = encoder.transform(['<EOS>'])[0]

group_counts = df.groupby('case:concept:name').size()
groups_to_clean = group_counts[(group_counts > max_length - 2)].index 

df_lstm_train = df_train[~df_train['case:concept:name'].isin(groups_to_clean)]
df_lstm_test = df_test

df_lstm_train = df_lstm_train[["case:concept:name","type_of_event", "days_since_last", "days_since_start", "dismissal_encoded", "in_debt"]]
df_lstm_test = df_lstm_test[["case:concept:name","type_of_event", "days_since_last", "days_since_start", "dismissal_encoded", "in_debt"]]    

classes = encoder.classes_
classes = encoder.transform(classes) 
onehot_encoder = OneHotEncoder(sparse_output = False) # we want an array of arrays, not a matrix object
onehot_encoder.fit(classes.reshape(-1, 1))

OneHotEncoder(sparse_output=False)

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Splits a dataframe with test/train data into prefixes and suffices - if test, put True as 2nd parameter
def prefixSuffix(df_lstm, test: bool = False): 
    all_events = df_lstm.groupby('case:concept:name')['type_of_event'].apply(list).tolist()
    times = df_lstm.groupby('case:concept:name')['days_since_last'].apply(list).tolist()
    
    window = 3 # the minimum prefix length INCLUDING the <SOS> token
    
    #Pad sequences with SOS and EOS tokens
    all_events = [[sos] + seq + [eos] for seq in all_events]
    times = [[float(0)] + time + [float(0)] for time in times]
    
    prefixes = []
    suffices = []
    prefixes_time = []
    suffices_time = []

    for seq, time_seq in zip(all_events, times): 
        for i in range(len(seq) - window):
            prefixes.append(seq[0:i+window])
            suffices.append(seq[i+window] if not test else seq[i+window:])
            prefixes_time.append(time_seq[0:i+window])
            suffices_time.append(time_seq[i+window] if not test else time_seq[i+window:])
    
    prefixes = pad_sequences(prefixes, maxlen=max_length - 1, padding='post', value=no_next, dtype=int)
    prefixes_time = pad_sequences(prefixes_time, maxlen=max_length - 1, padding='post', value=0, dtype=float)

    #One-hot encode suffices for training data
    if not test: 
        suffices = np.array(suffices)
        suffices_time = np.array(suffices_time)
        suffices = onehot_encoder.transform(suffices.reshape(-1,1))
        
    return prefixes, prefixes_time, suffices, suffices_time


In [12]:
prefixes, prefixes_time, suffices, suffices_time = prefixSuffix(df_lstm_train, False)

# We are giving events and times separately for easier debugging, but the model is concatenating them into 1 input layer
input_layer1 = Input(shape=(max_length - 1,))
input_layer2 = Input(shape=(max_length - 1,))
concatenated_inputs = Concatenate(axis=1)([input_layer1, input_layer2])
concatenated_inputs = Reshape((2, max_length - 1))(concatenated_inputs)

lstm_layer = LSTM(units=64, return_sequences=False)(concatenated_inputs)
lstm_layer = Dropout(0.2)(lstm_layer) # A slight dropout rate, because otherwise it tends to overfit 

event_type_output = Dense(14, activation='softmax', name='event_type')(lstm_layer)
timestamp_output = Dense(1, name='timestamp')(lstm_layer)  

# Define model
model = Model(inputs=[input_layer1, input_layer2], outputs=[event_type_output, timestamp_output])

# Compile model
model.compile(optimizer='adam', loss={'event_type': 'categorical_crossentropy', 'timestamp': 'mean_absolute_error'})

In [30]:
model.fit([prefixes, prefixes_time], [suffices, suffices_time], epochs=19, batch_size=64)

Epoch 1/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.6251
Epoch 2/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.3094
Epoch 3/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.2957
Epoch 4/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.2905
Epoch 5/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.2913
Epoch 6/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.2866
Epoch 7/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 0.2838
Epoch 8/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 0.2840
Epoch 9/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss: 0.2832
Epoch 10/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.2831
Epoch 11/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.2820
Epoch 12/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.2821
Epoch 13/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 0.2818
Epoch 14/19
4333/4333 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 0.2802
Epoch 15/19
433

In [31]:
max_prediction = max_length - 2 # set a cut-off point to avoid getting into an endless loop, value can be changed 
def predictSuffix(prefix, prefix_time, suffix): 
    prediction = model.predict([prefix, prefix_time])
    new_event = np.argmax(prediction[0], axis=1)[0]
    new_timestamp = prediction[1][0][0]
    
    if (new_event == eos):
        suffix.append([eos, 0.0]) #end of sequence token & padding time
        return suffix 

    suffix.append([new_event, new_timestamp])
    if np.isin(no_next, prefix[0]): #if there is still padding in the prefix
        index = (np.where(prefix[0] == no_next)[0][0]) #find the padding
        prefix[0, index] = new_event #replace it with the prediction
        prefix_time[0][index] = new_timestamp 
        return predictSuffix(prefix, prefix_time, suffix)
    else:
        if len(suffix) > max_prediction: # suspiciosly long prediction, cut it short 
            suffix.append([eos, 0.0])
            return suffix 
        else: 
            prefix.pop(0)
            prefix.append(new_event)
            prefix_time.pop(0)
            prefix_time.append(new_timestamp)
        return predictSuffix(prefix, prefix_time, suffix)

**Load test data**

In [13]:
prefixes, prefixes_time, suffices, suffices_time = prefixSuffix(df_lstm_test, True)

Example usage on a single trace from the test data

In [36]:
import random
ind = random.randint(0, len(prefixes) - 1) 
input1 = prefixes[ind].copy().reshape(1, -1)
input2 = prefixes_time[ind].copy().reshape(1, -1)
ind = 29

prediction = predictSuffix(input1, input2, []) 
event_prediction = [event[0] for event in prediction]
time_prediction = [time[1] for time in prediction]
actual_suffix = suffices[ind]
actual_times = suffices_time[ind] 

print('Predicted suffix:', encoder.inverse_transform(np.array(event_prediction).reshape(-1, 1)))
print('Actual suffix:', encoder.inverse_transform(actual_suffix))
print('The model is trained to predict how many days will have passed since the previous event:')
print('Predicted days:', scaler.inverse_transform(np.array(time_prediction).reshape(-1, 1)))
print('Actual days:', scaler.inverse_transform(np.array(actual_times).reshape(-1, 1))) 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted suffix: ['Insert Fine Notification' 'Add penalty' 'Send for Credit Collection'
 '<EOS>']
Actual suffix: ['Insert Fine Notification' 'Add penalty' 'Payment' '<EOS>']
The model is trained to predict how many days will have passed since the previous event:
Predicted days: [[ 18.58207213]
 [ 61.88227559]
 [419.45317924]
 [  0.        ]]
Actual days: [[ 11.]
 [ 60.]
 [101.]
 [  0.]]


C:\Users\20221807\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [34]:
def evaluate(sample):
    event_prediction = []
    for i in range(sample[0], sample[1]):
        suffix = []
        result = predictSuffix(prefixes[i].reshape(1, -1), prefixes_time[i].reshape(1, -1), suffix)
        event_prediction.append(result)
    return event_prediction

Evaluating on the entire test data takes very (very!) long, you can run on a sample by setting start and end index in an array and feeding it into evaluate()

In [18]:
#test_result = evaluate([0, len(prefixes)])

**Suffix Prediction Model Evaluation**

In [19]:
# #events are label-encoded, instead of 1h encoded, so first we need to deal with that:
# def convert_to_letters(sequence, mapping):
#     return [mapping[element] for element in sequence]

# int_to_letter = {i: chr(65 + i) for i in range(len(encoder.classes_))}

In [26]:
# import textdistance
# import statistics
# events_predicted = [[event[0] for event in suffix] for suffix in test_result] 
# times_predicted = [[event[1] for event in suffix] for suffix in test_result]
# times_predicted = pad_sequences(times_predicted, maxlen=14, value = 0, padding='post', dtype = float)
# suffices_time = pad_sequences(suffices_time, maxlen=14, value = 0, padding='post', dtype = float)

# #MAE
# decoded_prediction = scaler.inverse_transform(times_predicted)
# decoded_real = scaler.inverse_transform(suffices_time) 

# total_times_predicted = np.sum(decoded_prediction, axis=1) #predicted times until EOS
# total_times_real = np.sum(decoded_real, axis=1) #real times until EOS
# mae = mean_absolute_error(total_times_real, total_times_predicted) 
# print('MAE:', mae)

# #Represent each suffix as a string of unique letters 
# suffices_letters = [convert_to_letters(seq, int_to_letter) for seq in suffices]
# events_predicted_letters = [convert_to_letters(seq, int_to_letter) for seq in events_predicted]

# #Damerau-Levenstein (distance (not similarity), normalized)
# distances = []
# for actual, predicted in zip(suffices_letters, events_predicted_letters):
#     actual_str = "".join(actual)
#     predicted_str = "".join(predicted)

#     distance = textdistance.damerau_levenshtein.normalized_distance(actual_str, predicted_str)
#     distances.append(distance)
    
# print('DL Distance:', np.mean(distances))
# # #We take the mean and compute the similarity by subtracting it from 1
# print('DLS:', 1 - np.mean(distances))

MAE: 130.88949434431146
DL Distance: 0.1336055564607134
DLS: 0.8663944435392866


# Export

In [ ]:
df.to_csv('FinalTool23.csv')